In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
x = pickle.load(open("x.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [3]:
x.shape

(1721, 64, 64, 3)

In [4]:
def train_valid_test_split( x, y, train_size=0.7, test_size= 0.2, rand_seed=6789):
    valid_size = 1 - (train_size + test_size)
    
    x1, x_test, y1, y_test = train_test_split(x, y, test_size = test_size, random_state= rand_seed)

    x_train, x_valid, y_train, y_valid = train_test_split(x1, y1, test_size = float(valid_size)/(valid_size+ train_size))
    
    return (x_train, x_test, x_valid, y_train, y_test, y_valid)

In [5]:
# Split data

x_train, x_test, x_valid, y_train, y_test, y_valid = train_valid_test_split( x, y,
                                                                            train_size=0.7,
                                                                            test_size= 0.2,
                                                                            rand_seed=67)


In [6]:
generator = ImageDataGenerator(rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, 
                               channel_shift_range=10, horizontal_flip=True)

generator.fit(x_train)

In [7]:
cnn_model=Sequential()

cnn_model.add(Conv2D(input_shape=(64, 64, 3), filters=20, kernel_size=4, strides=2, padding='valid',
         activation='relu',  data_format='channels_last'))

cnn_model.add(Conv2D(filters=15, kernel_size=3, strides=1, padding='valid', activation='relu',
          data_format='channels_last'))

cnn_model.add(MaxPooling2D(pool_size=3, data_format='channels_last'))

cnn_model.add(Conv2D(filters=20, kernel_size=4, strides=2, padding='valid', activation='relu',
          data_format='channels_last'))

cnn_model.add(MaxPooling2D(pool_size=2, data_format='channels_last'))

# cnn_model.add(Conv2D(filters=15, kernel_size=2, strides=1, padding='valid', activation='relu',
#                       data_format='channels_last'))

# cnn_model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu',
#                       data_format='channels_last'))

# cnn_model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu',
#                      kernel_initializer='he_normal', data_format='channels_last'))

cnn_model.add(Flatten())

cnn_model.add(Dropout(0.2))

cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = cnn_model.fit(x_train, y_train, batch_size=32, epochs=100, verbose=1,
                validation_data=(x_valid, y_valid))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 1203 samples, validate on 173 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1203/1203 [==============================] - 1s 758us/sample - loss: 0.6923 - acc: 0.5287 - val_loss: 0.6931 - val_acc: 0.5087
Epoch 2/50
1203/1203 [==============================] - 0s 370us/sample - loss: 0.6912 - acc: 0.5436 - val_loss: 0.6938 - val_acc: 0.5087
Epoch 3/50
1203/1203 [==============================] - 0s 350us/sample - loss: 0.6903 - acc: 0.5428 - val_loss: 0.6944 - val_acc: 0.5087
Epoch 4/50
1203/1203 [==============================] - 0s 369us/sample - loss: 0.6901 - acc: 0.5428 - val_loss: 0.6942 - val_acc: 0.5087
Epoch 5/50
1203/1203 [==============================] - 0s 357us/sample - loss: 0.6901 - acc: 0.5428 - val_loss: 0.6939 - val_acc: 0.5087
Epoch 6/50
1203/1203 [======

In [47]:
model.evaluate(x_test, y_test)

2523/2523 [==============================] - 1s 232us/sample - loss: 0.6338 - acc: 0.6568


[0.6338149784247269, 0.65675783]

In [48]:
model.save("Trained_Model.model")